# Cross Lingual summarization CNN Daily Mail Results
We will try out the trained t5 network from the tpu

In [1]:
import tensorflow as tf
import pandas as pd
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
import time
from rouge_score import rouge_scorer
from rouge_score import scoring

In [2]:
if not tf.config.list_physical_devices('GPU'):
    print("Change runtime to \"GPU runtime\" for faster computations")

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

## Params

In [4]:
BATCH_SIZE = 8

SHUFFEL_SIZE = 1024

learning_rate = 3e-5

model_size = "t5-base"

MAX_ARTICLE_LEN = 512

MAX_HIGHLIGHT_LEN = 150

## Model

In [5]:
model = TFT5ForConditionalGeneration.from_pretrained(model_size)

task_specific_params = model.config.task_specific_params
if task_specific_params is not None:
    model.config.update(task_specific_params.get("summarization", {}))

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [6]:
val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

model.summary()

Model: "tf_t5for_conditional_generation"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
shared (TFSharedEmbeddings)  multiple                  24674304  
_________________________________________________________________
encoder (TFT5MainLayer)      multiple                  84954240  
_________________________________________________________________
decoder (TFT5MainLayer)      multiple                  113275008 
Total params: 222,903,552
Trainable params: 222,903,552
Non-trainable params: 0
_________________________________________________________________


In [9]:
ckpt_file = "../models/sueddeutsche_t5.ckpt"
model.load_weights(ckpt_file)

## Dataset
We will load the translated CNN Daily Mail dataset from the tfrecords files

In [10]:
class LanguageTokens:
    def __init__(self, tokenizer, tf_or_pt: str) -> None:
        super().__init__()
        self.en_de_prefix = tokenizer("summarize English to German: ", return_tensors=tf_or_pt).input_ids
        self.de_en_prefix = tokenizer("summarize German to English: ", return_tensors=tf_or_pt).input_ids
        self.en_en_prefix = tokenizer("summarize English to English: ", return_tensors=tf_or_pt).input_ids
        self.de_de_prefix = tokenizer("summarize German to German: ", return_tensors=tf_or_pt).input_ids

        if tf_or_pt == "tf":
            self.en_de_prefix = tf.reshape(self.en_de_prefix, (-1,))
            self.de_en_prefix = tf.reshape(self.de_en_prefix, (-1,))
            self.en_en_prefix = tf.reshape(self.en_en_prefix, (-1,))
            self.de_de_prefix = tf.reshape(self.de_de_prefix, (-1,))
        elif tf_or_pt == "pt":
            self.en_de_prefix = self.en_de_prefix.reshape(-1,)
            self.de_en_prefix = self.de_en_prefix.reshape(-1,)
            self.en_en_prefix = self.en_en_prefix.reshape(-1,)
            self.de_de_prefix = self.de_de_prefix.reshape(-1,)

        # check if last token is end of sequence token and remove it
        if self.en_de_prefix[-1] == 1:
            self.en_de_prefix = self.en_de_prefix[:-1]
            self.de_en_prefix = self.de_en_prefix[:-1]
            self.en_en_prefix = self.en_en_prefix[:-1]
            self.de_de_prefix = self.de_de_prefix[:-1]

        assert self.en_de_prefix.shape[0] == self.de_en_prefix.shape[0] == self.en_en_prefix.shape[0] == self.de_de_prefix.shape[0], "All perfixes must have the same size"
        self.prefix_size = self.en_de_prefix.shape[0]

In [11]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")
language_tokens = LanguageTokens(tokenizer, "tf")
prefix_size = language_tokens.prefix_size
prefix_size

5

In [12]:
import numpy as np
from os import listdir
MAX_ARTICLE_LEN = 512
MAX_HIGHLIGHT_LEN = 150
GLOBAL_BATCH_SIZE = 8

def get_tf_record_files(directory):
    file_list = []
    for item in listdir(directory):
        if item.split(".")[-1] == "tfrecord":
            file_list.append("{}/{}".format(directory, item))
    return file_list

def get_tfrecord_dataset(folder):
    features = {
        'ger_x': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-prefix_size], tf.int64),
        'ger_x_mask': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-prefix_size], tf.int64),
        'ger_y': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN], tf.int64),
        'ger_y_ids': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN], tf.int64),

        'en_x': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-prefix_size], tf.int64),
        'en_x_mask': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-prefix_size], tf.int64),
        'en_y': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN], tf.int64),
        'en_y_ids': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN], tf.int64),
    }
    
    dataset = tf.data.TFRecordDataset(get_tf_record_files(folder))

    # Taken from the TensorFlow models repository: https://github.com/tensorflow/models/blob/befbe0f9fe02d6bc1efb1c462689d069dae23af1/official/nlp/bert/input_pipeline.py#L24
    def decode_record(record, features):
        """Decodes a record to a TensorFlow example."""
        example = tf.io.parse_single_example(record, features)

        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.cast(t, tf.int32)
            example[name] = t
        return example

    def select_data_from_record(record):
        return [
            tf.concat([language_tokens.de_de_prefix, record['ger_x']], axis=0), tf.concat([tf.ones(prefix_size, dtype=tf.int32), record['ger_x_mask']], axis=0), record['ger_y'], record['ger_y_ids'],
            tf.concat([language_tokens.en_de_prefix, record['en_x']], axis=0), tf.concat([tf.ones(prefix_size, dtype=tf.int32), record['en_x_mask']], axis=0), record['ger_y'], record['ger_y_ids'],
            tf.concat([language_tokens.de_en_prefix, record['ger_x']], axis=0), tf.concat([tf.ones(prefix_size, dtype=tf.int32), record['ger_x_mask']], axis=0), record['en_y'], record['en_y_ids'],
            tf.concat([language_tokens.en_en_prefix, record['en_x']], axis=0), tf.concat([tf.ones(prefix_size, dtype=tf.int32), record['en_x_mask']], axis=0), record['en_y'], record['en_y_ids'],
        ]
    
    dataset = dataset.map(lambda record: decode_record(record, features))
    dataset = dataset.map(select_data_from_record)
    dataset = dataset.shuffle(100)
    return dataset.batch(GLOBAL_BATCH_SIZE)

root_folder = "../data/"
test_ds = get_tfrecord_dataset(root_folder + "sueddeutsche_test/")

In [13]:
def get_summaries(ds):
    for i in range(1,5):
        yield ds[(i-1)*4], ds[i*4-3], ds[i*4-2], ds[i*4-1]


for ds in test_ds.take(1):
    for i in get_summaries(ds):
        print(i[0].shape, i[1].shape, i[2].shape, i[3].shape)

(8, 512) (8, 512) (8, 150) (8, 150)
(8, 512) (8, 512) (8, 150) (8, 150)
(8, 512) (8, 512) (8, 150) (8, 150)
(8, 512) (8, 512) (8, 150) (8, 150)


## Evaluation
### Define Rouge Score

In [14]:
class RougeScore:
    '''
    mostly from https://github.com/google-research/text-to-text-transfer-transformer/blob/master/t5/evaluation/metrics.py 
    '''
    
    def __init__(self, score_keys=None)-> None:
        super().__init__()
        if score_keys is None:  
            self.score_keys = ["rouge1", "rouge2", "rougeLsum"]
        
        self.scorer = rouge_scorer.RougeScorer(self.score_keys)
        self.aggregator = scoring.BootstrapAggregator()
        
        
    @staticmethod
    def prepare_summary(summary):
            # Make sure the summary is not bytes-type
            # Add newlines between sentences so that rougeLsum is computed correctly.
            summary = summary.replace(" . ", " .\n")
            return summary
    
    def __call__(self, target, prediction):
        """Computes rouge score.''
        Args:
        targets: string
        predictions: string
        """

        target = self.prepare_summary(target)
        prediction = self.prepare_summary(prediction)
        
        self.aggregator.add_scores(self.scorer.score(target=target, prediction=prediction))

        return 
    
    def reset_states(self):
        self.rouge_list = []

    def result(self):
        result = self.aggregator.aggregate()
        
        for key in self.score_keys:
            score_text = "%s = %.2f, 95%% confidence [%.2f, %.2f]"%(
                key,
                result[key].mid.fmeasure*100,
                result[key].low.fmeasure*100,
                result[key].high.fmeasure*100
            )
            print(score_text)
        
        return {key: result[key].mid.fmeasure*100 for key in self.score_keys}

### Compute Summaries

In [23]:
predictions = []
start_time = time.time()

for i, ds_item in enumerate(test_ds): 
    for (input_ids, input_mask, y, y_ids) in get_summaries(ds_item):
        summaries = model.generate(
            input_ids=input_ids, 
            attention_mask=input_mask, 
            num_beams=4, 
            length_penalty=0.6,
            early_stopping=True, 
            max_length=150
        )

        articles = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in input_ids]

        pred = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summaries]
        real = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in y]
    
        for pred_sent, real_sent, article_sent in zip(pred, real, articles):
            predictions.append(str("article: " + article_sent + "\n\npred sentence: " + pred_sent + "\n\nreal sentence: " + real_sent))
    
    if (i % 10) == 0:
        elapsed = (time.time() - start_time) / 10
        print(i,": time genreate batch:", elapsed)
        start_time = time.time()
    if i > 20:
        # otherwise it will take ages
        break


# rouge_score.result()

0 : time genreate batch: 13.699205684661866
10 : time genreate batch: 104.2082646369934
20 : time genreate batch: 106.95739471912384


### Lets have a look at some of these predicted summaries

In [32]:
import numpy as np
len_predictions = len(predictions)

def get_random_prediction():
    return predictions[np.random.randint(len_predictions)]

In [33]:
print(get_random_prediction())

article: summarize English to German: Bodo Hechelhammer has meticulously investigated the career of multiple agent Heinz Felfe. The fact that someone was able to work for the BND and at the same time for the Soviets has mainly to do with the Nazi era. Bodo Hechelhammer spent eight years dealing with Heinz Felfe, namely "as a BND employee to understand why a former colleague committed treason and secrecy. Self-critically, "he writes in his acceptance speech," I have to admit that eight years was too many. " That is true, and again not true, because someone had to do the job, and no one is better equipped to do it than the former BND agent Hechelhammer, who now serves his agency as chief historian. As such, he has exclusive access to the BND archive, from which he was able to reconstruct how Felfe, a trained precision mechanic, worked first for the Reichssicherheitshauptamt (RSHA), then for the British secret service, then for the Soviet, finally for the West German foreign intelligence 

In [34]:
print(get_random_prediction())

article: summarize German to German: Norbert Walter-Borjans, 66 Jahre alt, hatte sich schon im Ruhestand eingerichtet. Der frühere Finanzminister aus Nordrhein-Westfalen fährt gerne Rad, er schreibt seiner Partei Steuerkonzepte. Er hat nach seinem Ende als Minister, 2017, ein Buch über seinen Kampf gegen Steuerhinterziehung geschrieben: "Steuern - Der große Bluff." Damit ist er durchs Land getourt. Es hätte so weitergehen können. Er war auf angenehme Art noch dabei und doch nicht mehr mittendrin. Dann aber kam mit dem Rückzug von Andrea Nahles von der Parteispitze alles in der SPD ins Rutschen. Jetzt ist Walter-Borjans zurück - voll und ganz. Er macht Wahlkampf für sich und seine Tandempartnerin Saskia Esken, 58, Digitalpolitikerin und Bundestagsabgeordnete aus dem Nordschwarzwald. Sie wollen an die Spitze. Sie haben gute Chancen, wie ihre Vorstellungsrunden bei den ersten Regionalkonferenzen zeigen. Sie kommen an bei den Mitgliedern, die diesmal über die Spitze entscheiden werden. Die

In [35]:
print(get_random_prediction())

article: summarize German to German: Nachdem US-Präsident Donald Trump im Februar dieses Jahres seine Absicht verkündet hatte, Kelly Craft als Botschafterin bei den Vereinten Nationen zu nominieren, dauerte es nicht lange, bis ein Fernsehinterview vom Oktober 2017 auftauchte, in dem Craft gefragt wurde, ob sie an den Klimawandel glaube. Sie sagte, dass sie "beide Seiten der Wissenschaft" respektiere, was insbesondere die Demokraten sehr erregte. Im Juli wurde sie dennoch vom Senat als künftige Botschafterin bestätigt, an diesem Dienstag wird sie vereidigt, gerade rechtzeitig vor der UN-Vollversammlung, die in der zweiten Septemberhälfte stattfindet. Was das Klima angeht, hat Craft ihre Aussage später modifiziert. Sie glaube durchaus, dass der Mensch Einfluss auf das Klima habe. Damit ging sie zumindest ein wenig auf Distanz zum Präsidenten, der öfter angemerkt hat, dass zwar das Klima sich wandele, das aber nicht notwendigerweise vom Menschen verursacht werde. In Crafts Fall wurde dies

In [36]:
print(get_random_prediction())

article: summarize English to German: The insolvent travel company Thomas Cook is warning its customers in Germany about an email scam. Customers are increasingly being asked to disclose sensitive information such as passport or credit card details. The emails sent look like official messages from the travel company, offering recipients a refund for their Thomas Cook trip. In a statement, the company said: "Thomas Cook has never sent emails of this nature to customers. Please ignore these mails and delete them. " All customers of the Thomas Cook organizers (Thomas Cook Signature, Thomas Cook Signature Finest Selection, Neckermann Reisen, ger Tours, Bucher Reisen and Air Marin) who wanted to travel up to and including 31 October could not travel due to insolvency law. Claims for reimbursement should be addressed to the insolvency administrator. For customers who are currently on vacation, hotel stays, as well as transfers and return flights as part of a package holiday are secured, the 

In [37]:
print(get_random_prediction())

article: summarize German to English: Vorgänger mit Nachfolgerin: Jean-Claude Juncker ging mit Ursula von der Leyen am Montag in Brüssel gemeinsam Mittagessen. Knapp zwei Monate, nachdem Ursula von der Leyen mit denkbar knapper Mehrheit vom Europaparlament zur ersten Präsidentin der EU-Kommission gewählt wurde, wird sie an diesem Dienstag mitteilen, wer in ihrem Team welches Portfolio übernehmen soll. Ihre politischen Leitlinien für die Zeit bis 2024 hat die CDU-Politikerin mit "Eine Union, die mehr erreichen will" überschrieben. Die Aufgabenverteilung im Kollegium wird andeuten, welche Prioritäten sie setzt und wer ihre Stützen sein werden. Völlig freie Hand hatte von der Leyen, die keine Spitzenkandidatin bei der Europawahl war, jedoch nicht. Im Juli bestimmten die Staats- und Regierungschefs nicht nur den Spanier Josep Borrell zum nächsten EU-Außenbeauftragten, sondern wiesen dem Sozialdemokraten Frans Timmermans und der dänischen Liberalen Margrethe Vestager herausgehobene Rollen a

In [38]:
for i in range(10):
    print(get_random_prediction())

article: summarize German to German: Der Tag der Preisverleihung war mit Bedacht gewählt: der Vorabend des katalanischen Nationalfestes, der Diada, zu dem sich auch an diesem Mittwoch wieder Hunderttausende zu einer Großkundgebung in Barcelona versammelt haben. Carola Rackete, Kapitänin des Rettungsschiffs Sea Watch 3, und scar Camps, der Gründer der spanischen Hilfsorganisation Proactiva Open Arms, wurden für ihren Einsatz zur Rettung von Migranten aus dem Mittelmeer vom katalanischen Regionalparlament geehrt. Die Laudatio hielt der berühmteste aller lebenden Katalanen, der Fußballtrainer Pep Guardiola, derzeit in Diensten des englischen Meisters Manchester City. Die Ehrenmedaillen verlieh kein geringerer als Parlamentspräsident Roger Torrent, führendes Mitglied der katalanischen Linksrepublikaner. Auch der rechtsliberale Regionalpräsident Quim Torra war anwesend. Es war also eine Veranstaltung erster Güte. Guardiola sagte: "Eine Welt, die nicht rettet, ist eine Welt, die untergeht, i

## Save results to text file

In [39]:
result_path = "../results/sueddeutsche_result.txt"
open(result_path, "w")
for pred in predictions:
    with open(result_path, "a") as file:
        file.write(pred + "\n")

## Load save File

In [40]:
data_points = []
result_path = "../results/sueddeutsche_result.txt"
file = open(result_path, "r")
for line in file:
    data_points.append(line)

In [49]:
data_points[0]

'article: summarize German to German: Was heute wichtig war - und was Sie auf SZ.de am meisten interessiert hat. Der Tag kompakt Britischer Parlamentspräsident Bercow kündigt Rücktritt an. Er werde spätestens am 31. Oktober aus privaten Gründen von seinem Amt zurücktreten. Premier Johnson will das Parlament am späten Montagabend nochmal über Neuwahlen abstimmen lassen. Danach will die britische Regierung das Unterhaus in eine fünfwöchige Zwangspause schicken, um eine Blockade von Johnsons Brexit-Plänen zu verhindern. Die aktuellen Entwicklungen im Live-Blog EXKLUSIV Ex-VW-Chef Winterkorn will Prozess verhindern. Winterkorn wie auch andere Angeschuldigte werfen der Staatsanwaltschaft schlampige Ermittlungen in der Abgasaffäre vor. Das Landgericht Braunschweig solle die Betrugsanklage zurückgeben. Von Klaus Ott Denkzettel für Kreml-Partei bei russischen Kommunalwahlen. Die Regierungspartei verliert teilweise dramatisch. Die ganz große Blamage für den Kreml bleibt jedoch aus - weil die me

In [42]:
class SummaryData():
    
    def __init__(self):
        self.language_tag = ''
        self.real_data = ''
        self.pred_data = ''        

In [52]:
count = 0
summary_data = SummaryData()
summary_data_list = []
for point in data_points:
    count += 1
    
    if count == 1:
        summary_data.language_tag = " ".join(point.split(" ")[2:5])
    elif count == 3:
        summary_data.pred_data = ": ".join(point.split(":")[1:])
    elif count == 5:
        summary_data.real_data = ": ".join(point.split(":")[1:])
        summary_data_list.append(summary_data)
        summary_data = SummaryData()
        count = 0
summary_data_list[0].__dict__

{'language_tag': 'German to German:',
 'real_data': ' Was heute wichtig war - und was Sie auf SZ.de am meisten interessiert hat.\n',
 'pred_data': ' Was heute wichtig war - und was Sie auf SZ.de am meisten interessiert hat. Der Tag kompakt Britischer Parlamentspräsident Bercow kündigt Rücktritt an.\n'}

In [44]:
from ast import literal_eval

results_en_trans = []
results_en_path = "../results/en_en_results"
file = open(results_en_path, "r")
for i, line in enumerate(file):
    results_en_trans.append(literal_eval(line))
    
results_ger_trans = []
results_ger_path = "../results/en_en_results"
file = open(results_ger_path, "r")
for i, line in enumerate(file):
    results_ger_trans.append(literal_eval(line))

In [58]:
rouge_scores_dict = dict()
rouge_scores_dict['English to English:'] = RougeScore()
# rouge_scores_dict['en_to_en_trans'] = RougeScore()

rouge_scores_dict['English to German:'] = RougeScore()
rouge_scores_dict['German to English:'] = RougeScore()
rouge_scores_dict['German to German:'] = RougeScore()
# rouge_scores_dict['ger_to_ger_trans'] = RougeScore()

rouge_scores_dict

{'English to English:': <__main__.RougeScore at 0x7f616c4b1278>,
 'English to German:': <__main__.RougeScore at 0x7f616c4b1e80>,
 'German to English:': <__main__.RougeScore at 0x7f616c4b15c0>,
 'German to German:': <__main__.RougeScore at 0x7f616c4b1588>}

In [59]:
for summary_data in summary_data_list:
    rouge_scores_dict[summary_data.language_tag](summary_data.real_data, summary_data.pred_data)
    

In [60]:
results_ger_trans[0]

{'id': 24,
 'item_id': 1,
 'language_tag': 'en_to_en',
 'real_data': 'Beatrice seen watching race on terrace with the Gulf states Crown Prince . Marks 13th holiday since November last year, and fourth in a month . Princess quit her job at Sony Pictures in New York before Christmas . Despite that she is described as working full-time on her fathers website .\n',
 'pred_data': 'Princess Beatrice spotted at Bahrain Grand Prix with long-term boyfriend Dave Clark . Onlooker said 26-year-old was walking behind the Crown Prince of Bahrain . Princes regime accused of violently repressing pro-democracy protests . Sir Jackie Stewart and comedian Rory Bremner watched race from the tower .\n',
 'translated': 'Prinzessin Beatrice beim Großen Preis von Bahrain mit ihrem langjährigen Freund Dave Clark.Beobachter sagten, der 26-Jährige sei hinter dem Kronprinzen von Bahrain hergelaufen.Prinzen-Regime beschuldigt, prodemokratische Proteste gewaltsam zu unterdrücken.Sir Jackie Stewart und Komiker Rory B

In [61]:
for key, rouge_score_item in rouge_scores_dict.items():
    print(key, rouge_score_item.result())
    print()

rouge1 = 53.72, 95% confidence [49.52, 58.19]
rouge2 = 39.75, 95% confidence [34.36, 45.30]
rougeLsum = 49.07, 95% confidence [44.59, 53.65]
English to English: {'rouge1': 53.721893810948714, 'rouge2': 39.74677978368468, 'rougeLsum': 49.06511063004578}

rouge1 = 33.73, 95% confidence [29.77, 37.71]
rouge2 = 18.80, 95% confidence [15.47, 22.33]
rougeLsum = 29.32, 95% confidence [25.38, 33.12]
English to German: {'rouge1': 33.72740943495188, 'rouge2': 18.802521727543457, 'rougeLsum': 29.320106813110165}

rouge1 = 48.19, 95% confidence [45.15, 51.24]
rouge2 = 29.66, 95% confidence [26.75, 32.96]
rougeLsum = 43.08, 95% confidence [39.95, 46.24]
German to English: {'rouge1': 48.19478204878956, 'rouge2': 29.658696858972462, 'rougeLsum': 43.0783713149098}

rouge1 = 41.07, 95% confidence [35.64, 46.22]
rouge2 = 30.95, 95% confidence [25.01, 36.99]
rougeLsum = 37.47, 95% confidence [31.87, 43.37]
German to German: {'rouge1': 41.06746208280935, 'rouge2': 30.94555875619524, 'rougeLsum': 37.465862

# Special Example

In [29]:
input_text = "In a world where we have to read and understand a lot of documents automatic text summarization has an obvious demand. To have the option to get a brief summary of a text in your language can be very useful. But would it not be even more useful, if you could have the option to get a summary of a text in a language, you do not understand, in your wished language? Sometimes we don’t want an exact translation, sometimes we just want to know a brief overview of a text in a language we don’t understand. That is the case where Cross-Lingual summarization would be prefered over normal translation. Cross-Lingual summarization can give you a short overview of a text in a language you do not understand."

In [30]:
input_text_test = "The dramatic growth of data on the internet leads to the need to automatically process and understand the data. A big part of the data is text data in many languages. This overwhelming amount of information causes a demand for automatic text summarization and other Natural Language Processing (NLP) taks.\n The Field of NLP had some high points in the last couple of years, when the field got revolutionized by Neural Language Models. With publications like Attention is all you need \cite{vaswani2017attention} or GPT3\cite{brown2020language} the limits of the field are pushed even further. Because of large data corpuses, scraped from the internet, and advanced models which contain up to 175 Billion parameters like the GPT3 it is possible to generate text, answer questions, summarize text, translate or many other things.\n In this master thesis we will take a deeper look into summarization. There are extractive and abstractive techniques to summarization. The extractive summary technique tries to find subsets of sentences, which representante the original text well and uses them to summarize the original text \cite{allahyari2017text}. The abstractive technique uses advanced language models to generate a new text, which should be much shorter than the original one, contain all the key information and preserve the overall meaning.\n In this master thesis we will use the abstractive technique, because it is closer to a human-like interpretation. It combines the ability to understand what the context of a given text is and the ability to generate fluent and grammatically correct text to that given context.\n The summarization in one language is an interesting topic but cross lingual approach is even more interesting. The goal of Cross Lingual Summarization is to summarize a text from one language into another language. It combines the ability to summarize and the ability to translate. Where state of the art models perform well on normal summarization, it will be interesting to see how well they perform doing Cross-Lingual Summarization."

In [31]:
x = tokenizer.encode_plus("summarize: en_to_ger " + input_text, max_length=512, return_tensors="tf", padding='max_length', truncation=True)
input_ids = tf.reshape(x['input_ids'], (1,-1))
attention_mask = tf.reshape(x['attention_mask'], (1,-1))

In [32]:
print(input_ids.shape, attention_mask.shape)

(1, 512) (1, 512)


In [33]:
summaries = model.generate( 
    input_ids=input_ids, 
    attention_mask=attention_mask
)

In [34]:
summaries

<tf.Tensor: shape=(1, 74), dtype=int32, numpy=
array([[    0,    86,   645,  3779,     6,    16,    74,   558,  2584,
            3, 20127,    15,   110,    35,    64, 19163,  3766,     6,
          229, 15820,    15,  5027,     7,    63,    29, 19712,   266,
        30410,    15,  5222,  6367,     5,  4098,  6199,     3,    15,
            7,   311, 30001,    49,     6,  1301,   292,    67,  7251,
         8219,    29,     6,   266, 11068, 14449,   266,     7,  5027,
           15,     7,    16,   645, 16933,   170,  8837,     6,    67,
          292,   311, 19163,     6,    16,  1197,    52, 24054,    29,
        16933,    58]], dtype=int32)>

In [35]:
tokenizer.decode(summaries[0])

'<pad> In einer Welt, in der wir viele Dokumente lesen und verstehen müssen, ist automatische Textsynthese eine offensichtliche Forderung. Aber wäre es nicht nützlicher, wenn Sie die Möglichkeit hätten, eine Zusammenfassung eines Textes in einer Sprache zu bekommen, die Sie nicht verstehen, in Ihrer gewünschten Sprache?'